# Stateful Long-Short strategy 

In [4]:
# Import necessary libraries.
import xarray as xr
import pandas as pd
import numpy as np
import datetime
import os 
import pickle
import qnt.data    as qndata  
import qnt.output as qnout   
import qnt.backtester as qnbt 
import qnt.stats   as qnstats 
import qnt.graph   as qngraph 
import qnt.ta      as qnta   
import qnt.xr_talib as xr_talib   
import qnt.state as qnstate 
import qnt.exits as qnte

# Strategy Logic

- long_signal: Generated when the 51-day EMA is above the 87-day EMA by 1.5*ATR or more.

- long_signal_2: Generated when the 14-day SMA of TR divided by close  is below it's  28-day SMA  multiplied by 0.9                              indicating  a period of lower volatility

- exit1: Closes the long position when the  close is under the 27-day EMA

- short_signal: Generated when the 7-day EMA is below the 82-day EMA by 1.5*ATR or more and close is below 3-day EMA and RSI is                 above 60 
 
We will add all the signals up. In this case the exit1 is made to exit long positions, so we will multiply it with long         signals. These exits are a part of our signal, and we will further filter it with exits that depend on our position.

- signal_tp: Generate exit signal from long position when close price move upwards from the entry opening price by 6.35 ATR                  
- signal_sl: Generate exit signal from long position when close price move downwards from the entry opening price by 2.9 ATR                
- signal_dc: Generate exit signal from long position when 151 bars pass since entering long position

- signal_tp_short: Generate exit signal from short position when close price move downwards from the entry opening price by                        0.72 ATR                

- signal_sl_short: Generate exit signal from short position when close price move upwards from the entry opening price by                          0.47 ATR  

- signal_dc_short: Generate exit signal from short position when 9 bars pass since entering short position


#### Position Sizing

Positions are sized based on the ATR percentage ```weights = (entry/atr_perc)``` in the code. This is a risk management method that invests more in stable periods of low volatility and less in high volatility periods.


The weights are updated by multiplying them with all the exit signals (signal_tp, signal_sl, dsignal_dc, signal_tp_short, signal_sl_short and signal_dc_short ). This effectively exits positions when any exit condition occurs.

#### State Management
The state is updated with the new weights and written back to ensure persistence across function calls. In this implementation, positions are forwarded every day until an exit is hit. The system will stay in position until an exit happens, after which it looks for  entry signal again.

**Note: Exit functions only work properly with the multi-pass backtester due to requiring previous state information. 
Make sure to apply at least one exit to your long and short signals, to avoid them being held indefinitely.**

In [6]:
def strategy(data, state):
    
    #Tehnical indicators
    close = data.sel(field="close")
    open_ = data.sel(field="open")
    high=data.sel(field='high')
    low=data.sel(field='low')
    atr14 = qnta.atr(data.sel(field='high'), data.sel(field='low'), data.sel(field='close'), 14)
    last_atr = atr14.isel(time=-1)
    ema87=qnta.ema(close,87)
    ema51=qnta.ema(close,51)
    ema27=qnta.ema(close,27)
    tr = qnta.tr(high, low, close)
    vol_osc= qnta.sma(tr/close, 14)
    rsi_14 = qnta.rsi(close, 14)
    atr_perc = xr.where(atr14/close > 0.01, atr14/close, 0.01)
    ema7=qnta.ema(close,7)
    ema82=qnta.ema(close,82)
    ema3=qnta.ema(close,3)
    
    
    
    
    if state is None:
        state = {
            "weights": xr.zeros_like(close),
            "open_price": xr.full_like(data.isel(time=-1).asset, np.nan, dtype=int),
            "holding_time": xr.zeros_like(data.isel(time=-1).asset, dtype=int),
            }
        qnstate.write(state)
    weights_prev = state['weights']
    #To reuse the template, define your trading signals here ---------------------
    long_signal=xr.where(ema51>ema87+1.5*atr14,1,0)
    long_signal_2=xr.where(vol_osc<0.9*qnta.sma(vol_osc,28),1,0)
    exit1=xr.where(close<ema27 ,0,1)
    short_signal=xr.where((ema7<ema82-1.5*atr14) & (close<ema3)&(rsi_14>60)  ,-1,0)
    entry_signal=short_signal+long_signal*long_signal_2*exit1
    entry_signal = entry_signal/atr_perc

    # ----------------------------------------------------------------------------
    
    #Keeping track of the previous position
    weights_prev, entry_signal = xr.align(weights_prev, entry_signal, join='right')
    weights=xr.where(entry_signal==0,weights_prev.shift(time=1),entry_signal)
    weights=weights.fillna(0)
    
    #Define additional exit parameters here----------------------------------
    open_price = qnte.update_open_price(data, weights, state)
    signal_tp = qnte.take_profit_long_atr(data, weights, open_price, last_atr, atr_amount = 6.35) 
    signal_sl = qnte.stop_loss_long_atr(data, weights, open_price, last_atr, atr_amount = 2.9) 
    signal_dc = qnte.max_hold_long(weights, state, max_period = 151)
    signal_dc_short = qnte.max_hold_short(weights, state, max_period = 9)
    signal_tp_short = qnte.take_profit_short_atr(data, weights, open_price, last_atr, atr_amount = 0.72)
    signal_sl_short = qnte.stop_loss_short_atr(data, weights, open_price, last_atr, atr_amount = 0.47)
    weights = weights * signal_tp * signal_sl * signal_dc* signal_tp_short * signal_sl_short * signal_dc_short
    #cutting weigths to max absolute value 0.1
    weights=xr.where(abs(weights) > 0.1, np.sign(weights)*0.1, weights)
    #------------------------------------------------------------------------
    
    state['weights'] = weights
    return weights, state

weights, state = qnbt.backtest(
    competition_type="stocks_nasdaq100", 
    lookback_period=365,  # lookback in calendar days
    start_date="2006-01-01",
    strategy=strategy,
    analyze=True,
    build_plots=True,
    collect_all_states=False # if it is False, then the function returns the last state, otherwise - all states
)

Run last pass...
Load data...


100% (39443 of 39443) |##################| Elapsed Time: 0:00:00 Time:  0:00:00
100% (4004752 of 4004752) |##############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 1/1 2s
Data loaded 2s
Run strategy...
State saved.
Load data for cleanup...


100% (641412 of 641412) |################| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 1/1 2s
Data loaded 2s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
WARNING! Strategy trades non-liquid assets.
Fix liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.
Write result...
Write output: /root/fractions.nc.gz
State saved.
---
Run first pass...
Load data...


...

100% (13001792 of 13001792) |############| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 6/7 5s


100% (970232 of 970232) |################| Elapsed Time: 0:00:00 Time:  0:00:00


fetched chunk 7/7 5s
Data loaded 5s
Output cleaning...
fix uniq
ffill if the current price is None...
Check liquidity...
WARNING! Strategy trades non-liquid assets.
Fix liquidity...
Ok.
Check missed dates...
Ok.
Normalization...
Output cleaning is complete.
Write result...
Write output: /root/fractions.nc.gz
State saved.
---
Analyze results...
Check...
Check liquidity...
Ok.
Check missed dates...
Ok.
Check the sharpe ratio...
Period: 2006-01-01 - 2024-07-02
Sharpe Ratio = 0.7418361996096839
Ok.
---
Align...
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Select the asset (or leave blank to display the overall stats):


interactive(children=(Combobox(value='', description='asset', options=('', 'NAS:AAL', 'NAS:AAPL', 'NAS:ABNB', …